In [1]:
%pip install openpyxl
import pandas as pd
import numpy as np
import re
import difflib
from IPython.utils import io
from IPython.display import display

Note: you may need to restart the kernel to use updated packages.


Importer le notebook Données stations de métro 2022

In [2]:
try:
    with io.capture_output():
        %run -i "Données stations de métro 2022.ipynb"
except Exception as e:
    display(e)

Importer les données disponibles sur le site https://data.ratp.fr. \
Ce jeu de données présente le trafic des entrants directs sur le réseau ferré RATP en 2021.

In [3]:
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print("Tableau importé : trafic des entrants directs sur le réseau ferré RATP en 2021")
display(tableau_fréquentation)

Tableau importé : trafic des entrants directs sur le réseau ferré RATP en 2021


Rang Réseau               Station    Trafic Correspondance_1  \
0       3  Métro          GARE DE LYON  28640475                1   
1      12  Métro            BELLEVILLE   7314438                2   
2      32  Métro                 OPERA   5193831                3   
3      58  Métro  SAINT-MANDE-TOURELLE   3944640                1   
4      59  Métro                 HOCHE   3928404                5   
..    ...    ...                   ...       ...              ...   
366    39    RER    FONTENAY-SOUS-BOIS   2121720                A   
367    40    RER         SUCY-BONNEUIL   1993503                A   
368    58    RER            LE GUICHET    530706                B   
369    65    RER        PARC DE SCEAUX    383626                B   
370    66    RER        LA HACQUINIERE    285626                B   

    Correspondance_2 Correspondance_3  Correspondance_4  Correspondance_5  \
0                 14                A               NaN               NaN   
1                 11              NaN               NaN               NaN   
2                  7                8               NaN               NaN   
3                NaN              NaN               NaN               NaN   
4                NaN              NaN               NaN               NaN   
..               ...              ...               ...               ...   
366              NaN              NaN               NaN               NaN   
367              NaN              NaN               NaN               NaN   
368              NaN              NaN               NaN               NaN   
369              NaN              NaN               NaN               NaN   
370              NaN              NaN               NaN               NaN   

                  Ville  Arrondissement pour Paris  
0                 Paris                       12.0  
1                 Paris                       10.0  
2                 Paris                        9.0  
3           Saint Mandé                        NaN  
4                Pantin                        NaN  
..                  ...                        ...  
366  Fontenay-sous-Bois                        NaN  
367        Sucy-en-Brie                        NaN  
368               Orsay                        NaN  
369              Antony                        NaN  
370    Bures sur Yvette                        NaN  

[371 rows x 11 columns]

Nettoyer le tableau

In [4]:
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
tableau_fréquentation = tableau_fréquentation.drop_duplicates(subset='Station')

colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

display(tableau_fréquentation)

Rang               Station    Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  28640475                1               14   
1      12            BELLEVILLE   7314438                2               11   
2      32                 OPERA   5193831                3                7   
3      58  SAINT-MANDE-TOURELLE   3944640                1              nan   
4      59                 HOCHE   3928404                5              nan   
..    ...                   ...       ...              ...              ...   
350   272         SULLY-MORLAND   1124169                7              nan   
351   275                EUROPE   1106459                3              nan   
352   283          CAMPO-FORMIO   1016150                5              nan   
353   298         SAINT-FARGEAU    472258             3bis              nan   
354   300       PORTE D'AUTEUIL    375748               10              nan   

    Correspondance_3 Correspondance_4 Correspondance_5  
0                                 nan              nan  
1                nan              nan              nan  
2                  8              nan              nan  
3                nan              nan              nan  
4                nan              nan              nan  
..               ...              ...              ...  
350              nan              nan              nan  
351              nan              nan              nan  
352              nan              nan              nan  
353              nan              nan              nan  
354              nan              nan              nan  

[305 rows x 8 columns]

Créer une colonne comptant le nombre de correspondances à chaque station et diviser les valeurs de la colonne "Trafic" par le nombre de correspondances 

In [5]:
# Ajouter la colonne 'nb_correspondance'
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].apply(lambda row: sum(pd.notna(value) and value != 'nan' for value in row), axis=1)

# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']

display(tableau_fréquentation)

Rang               Station     Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  9546825.0                1               14   
1      12            BELLEVILLE  3657219.0                2               11   
2      32                 OPERA  1731277.0                3                7   
3      58  SAINT-MANDE-TOURELLE  3944640.0                1              nan   
4      59                 HOCHE  3928404.0                5              nan   
..    ...                   ...        ...              ...              ...   
350   272         SULLY-MORLAND  1124169.0                7              nan   
351   275                EUROPE  1106459.0                3              nan   
352   283          CAMPO-FORMIO  1016150.0                5              nan   
353   298         SAINT-FARGEAU   472258.0             3bis              nan   
354   300       PORTE D'AUTEUIL   375748.0               10              nan   

    Correspondance_3 Correspondance_4 Correspondance_5  nb_correspondance  
0                                 nan              nan                  3  
1                nan              nan              nan                  2  
2                  8              nan              nan                  3  
3                nan              nan              nan                  1  
4                nan              nan              nan                  1  
..               ...              ...              ...                ...  
350              nan              nan              nan                  1  
351              nan              nan              nan                  1  
352              nan              nan              nan                  1  
353              nan              nan              nan                  1  
354              nan              nan              nan                  1  

[305 rows x 9 columns]

Créer un dictionnaire contenant des données d'affluence pour chaque station de chaque ligne 

In [6]:
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14] 

# Créer des tableaux pour chaque ligne de métro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in x.values, axis=1)]
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows
    
# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)
    tableau = tableau.drop_duplicates(subset='Station')
    tableaux[tableau_nom] = tableau.drop_duplicates(subset='Station')  # Supprimer les doublons de la colonne 'Station'

Ajouter pour chaque tableau la fréquentation totale et la part du trafic de la station par rapport à toute la ligne, et fusionner les données

In [7]:
# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

Remplir les valeurs manquantes dans la colonne 'LIBELLE_ARRET' des tableaux en trouvant la correspondance la plus proche dans le dataframe moyenne_par_arret pour chaque ligne où cette valeur est manquante.

In [8]:
# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match

Fusionner les données avec le tableau moyenne_par_arret sur la colonne 'LIBELLE_ARRET' pour récupérer les valeurs de 'NB_VALD' associées à chaque 'LIBELLE_ARRET'

In [9]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald


Fusionner chaque tableau avec pivot_table (créé à partir du notebook Données stations de métro 2022) en utilisant la colonne 'LIBELLE_ARRET' comme clé de fusion

In [10]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

Supprimer les lignes en double basées sur la colonne 'Station' de chaque tableau

In [11]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les doublons de stations
    current_table = current_table.drop_duplicates(subset='Station')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table


Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' de chaque tableau

In [12]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

Pour chaque tableau, céer une nouvelle colonne moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne en divisant deux colonnes existantes (moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station et nb_correspondance)

In [13]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table


Pour chaque tableau, créer une nouvelle colonne moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction en divisant deux colonnes existantes (moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station, nb_correspondance) et en divisant le résultat par 2 (deux sens : aller / retour)

In [14]:

for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table


Pour chaque tableau, créer une nouvelle colonne nombre_validations_totales_pour_la_ligne en sommant les valeurs de la colonne moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne 

In [15]:

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table


Réorganiser le tableau

In [16]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic', 
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne','fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]


Renommer les colones pour plus de visibilité 

In [17]:
# Nouveaux noms des colonnes
nouveaux_noms_colonnes = {
    'Station': 'Station',
    'Trafic': 'Trafic annuel dans la station',
    'Part du trafic de la station par rapport à toute la ligne': 'Part de la station dans le trafic annuel de la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station': 'Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne': 'Moyenne de la fréquentation journalière de chaque correspondance de la station en 2022, sur la période des JO',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction': 'Moyenne de la fréquentation journalière de chaque correspondance, pour chaque direction en 2022, sur la période des JO',
    'Correspondance_1': 'Correspondance 1',
    'Correspondance_2': 'Correspondance 2',
    'Correspondance_3': 'Correspondance 3',
    'Correspondance_4': 'Correspondance 4',
    'Correspondance_5': 'Correspondance 5',
    'nb_correspondance': 'Nombre de correspondances de la station',
    'nombre_validations_totales_pour_la_ligne': 'Nombre de validations annuelles sur la ligne',
    'fréquentation_totale_ligne': 'Fréquentation totale de la ligne',
    '00:00-01:00': '00:00-01:00',
    '01:00-02:00': '01:00-02:00',
    '02:00-03:00': '02:00-03:00',
    '03:00-04:00': '03:00-04:00',
    '04:00-05:00': '04:00-05:00',
    '05:00-06:00': '05:00-06:00',
    '06:00-07:00': '06:00-07:00',
    '07:00-08:00': '07:00-08:00',
    '08:00-09:00': '08:00-09:00',
    '09:00-10:00': '09:00-10:00',
    '10:00-11:00': '10:00-11:00',
    '11:00-12:00': '11:00-12:00',
    '12:00-13:00': '12:00-13:00',
    '13:00-14:00': '13:00-14:00',
    '14:00-15:00': '14:00-15:00',
    '15:00-16:00': '15:00-16:00',
    '16:00-17:00': '16:00-17:00',
    '17:00-18:00': '17:00-18:00',
    '18:00-19:00': '18:00-19:00',
    '19:00-20:00': '19:00-20:00',
    '20:00-21:00': '20:00-21:00',
    '21:00-22:00': '21:00-22:00',
    '22:00-23:00': '22:00-23:00',
    '23:00-00:00': '23:00-00:00',
    'Rang': 'Rang'
}

# Afficher chaque tableau final avec le nouvel ordre des colonnes et les nouveaux noms
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    # Renommer les colonnes
    tableau = tableau.rename(columns=nouveaux_noms_colonnes)
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")


Tableau: ligne numéro[1]


Station  Trafic annuel dans la station  \
0                gare de lyon                   9.546825e+06   
2        saint-mande-tourelle                   3.944640e+06   
3                      louvre                   1.869612e+06   
4          porte de vincennes                   5.446602e+06   
5               porte maillot                   4.138301e+06   
7        chateau de vincennes                   3.617738e+06   
8                    chatelet                   1.670159e+06   
9             pont de neuilly                   4.809503e+06   
10    esplanade de la defense                   4.708183e+06   
11                 saint-paul                   4.295823e+06   
12   charles de gaulle-etoile                   1.430554e+06   
14                les sablons                   3.954920e+06   
15                 la defense                   9.256802e+06   
16             hotel de ville                   3.625864e+06   
17               palais-royal                   2.411300e+06   
18                   concorde                   1.133740e+06   
19                  argentine                   2.079212e+06   
20  champs-elysees-clemenceau                   9.545025e+05   
21                  tuileries                   1.859552e+06   
22                   bastille                   2.689748e+06   
23      franklin d. roosevelt                   3.086915e+06   
24                     nation                   1.512699e+06   
25            reuilly-diderot                   2.290046e+06   
26                   george v                   3.842260e+06   
27                    berault                   2.106827e+06   

    Part de la station dans le trafic annuel de la ligne  \
0                                            0.110646      
2                                            0.045718      
3                                            0.021669      
4                                            0.063125      
5                                            0.047962      
7                                            0.041929      
8                                            0.019357      
9                                            0.055741      
10                                           0.054567      
11                                           0.049788      
12                                           0.016580      
14                                           0.045837      
15                                           0.107285      
16                                           0.042023      
17                                           0.027947      
18                                           0.013140      
19                                           0.024098      
20                                           0.011063      
21                                           0.021552      
22                                           0.031174      
23                                           0.035777      
24                                           0.017532      
25                                           0.026541      
26                                           0.044531      
27                                           0.024418      

    Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)  \
0                                         5298.853444                                                                 
2                                         1321.871981                                                                 
3                                          654.393116                                                                 
4                                         1720.644324                                                                 
5                                         1781.472869                                                                 
7                                         1720.644324                             



Tableau: ligne numéro[2]


Station  Trafic annuel dans la station  \
0                 belleville                   3.657219e+06   
1             colonel fabien                   3.043606e+06   
2               menilmontant                   2.847264e+06   
3                    blanche                   2.167570e+06   
4           philippe auguste                   1.262653e+06   
5                    monceau                   1.148677e+06   
6               place clichy                   2.580966e+06   
7                la chapelle                   4.855531e+06   
8                     jaures                   1.351820e+06   
9                      avron                   1.345521e+06   
10                stalingrad                   1.641528e+06   
11  charles de gaulle-etoile                   1.430554e+06   
13             pere lachaise                   1.732654e+06   
14                 couronnes                   2.151515e+06   
15            porte dauphine                   2.021656e+06   
16                      rome                   1.696331e+06   
17               victor hugo                   2.801041e+06   
18           alexandre dumas                   2.476608e+06   
19                    nation                   1.512699e+06   
20       barbes-rochechouart                   2.695470e+06   
21                  villiers                   1.793188e+06   
22                   pigalle                   1.750916e+06   
23                    anvers                   3.420749e+06   
24                    ternes                   2.292256e+06   
25                courcelles                   1.583429e+06   
26               funiculaire                   1.528671e+06   

    Part de la station dans le trafic annuel de la ligne  \
0                                            0.064399      
1                                            0.053594      
2                                            0.050137      
3                                            0.038168      
4                                            0.022234      
5                                            0.020227      
6                                            0.045447      
7                                            0.085500      
8                                            0.023804      
9                                            0.023693      
10                                           0.028905      
11                                           0.025190      
13                                           0.030510      
14                                           0.037885      
15                                           0.035599      
16                                           0.029870      
17                                           0.049323      
18                                           0.043610      
19                                           0.026637      
20                                           0.047464      
21                                           0.031576      
22                                           0.030831      
23                                           0.060235      
24                                           0.040364      
25                                           0.027882      
26                                           0.026918      

    Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)  \
0                                         2329.793584                                                                 
1                                          961.150485                                                                 
2                                          885.649272                                                                 
3                                          628.054612                                                                 
4                                          385.908318                                                  



Tableau: ligne numéro[3]


...

Station  Trafic annuel dans la station  \
0                   rue du bac                   1.423364e+06   
1                       rennes                   8.613340e+05   
2       montparnasse-bienvenue                   5.101806e+06   
4                    madeleine                   1.776976e+06   
5              sevres-babylone                   1.696252e+06   
6                mairie d'issy                   2.874138e+06   
7        notre-dame-de-lorette                   1.893828e+06   
8          porte de versailles                   3.268157e+06   
9                    vaugirard                   2.482886e+06   
10                 marx dormoy                   2.425928e+06   
11             front populaire                   1.948542e+06   
12        porte de la chapelle                   1.866281e+06   
13                 volontaires                   1.734848e+06   
14                   solferino                   1.269143e+06   
15                   falguiere                   6.502910e+05   
16                    concorde                   1.133740e+06   
17                     pasteur                   1.513143e+06   
18                    abbesses                   1.513884e+06   
19  trinite-d'estienne d'orves                   1.051982e+06   
20               saint-georges                   7.515550e+05   
21         assemblee nationale                   6.115120e+05   
22                saint-lazare                   6.625677e+06   
24       marcadet-poissonniers                   1.991002e+06   
25                  convention                   3.734750e+06   
26                     pigalle                   1.750916e+06   
27               jules joffrin                   2.854819e+06   
28             corentin celton                   2.504682e+06   
29        lamarck-caulaincourt                   1.875717e+06   
30       notre-dame-des-champs                   1.487256e+06   

    Part de la station dans le trafic annuel de la ligne  \
0                                            0.023459      
1                                            0.014196      
2                                            0.084085      
4                                            0.029287      
5                                            0.027957      
6                                            0.047370      
7                                            0.031213      
8                                            0.053864      
9                                            0.040921      
10                                           0.039983      
11                                           0.032115      
12                                           0.030759      
13                                           0.028593      
14                                           0.020917      
15                                           0.010718      
16                                           0.018686      
17                                           0.024939      
18                                           0.024951      
19                                           0.017338      
20                                           0.012387      
21                                           0.010079      
22                                           0.109201      
24                                           0.032815      
25                                           0.061554      
26                                           0.028858      
27                                           0.047051      
28                                           0.041281      
29                                           0.030914      
30                                           0.024512      

    Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)  \
0                                          511.815741                                                                 
1                                



Tableau: ligne numéro[13]


Station  Trafic annuel dans la station  \
0       saint-denis-universite                      3569990.0   
1               les courtilles                      2802689.0   
2          porte de saint-ouen                      2710638.0   
3              porte de clichy                      2639248.5   
4                 place clichy                      2580966.0   
5              porte de vanves                      3395358.0   
6                      pernety                      2173567.0   
7             carrefour pleyel                      1535528.0   
8                 gabriel peri                      3790573.0   
9                   miromesnil                      1823627.5   
10                   invalides                      1741040.0   
12  saint-denis-porte de paris                      3355836.0   
13                       duroc                      1322532.0   
14                       gaite                      1644148.0   
15                    brochant                      1543298.0   
16       saint-francois-xavier                      1213378.0   
17                       liege                      1143956.0   
18    basilique de saint-denis                      3991395.0   
19                   plaisance                      3521753.0   
20  malakoff-plateau de vanves                      2242320.0   
21                  guy moquet                      2205313.0   
22   champs-elysees-clemenceau                       954502.5   
23                les agnettes                      1890356.0   
24                  la fourche                      1829271.0   
25                     varenne                       782697.0   
26         chatillon-montrouge                      5034012.0   
27        mairie de saint-ouen                      2415405.0   
28            mairie de clichy                      4043071.0   
29                   garibaldi                      1965002.0   
30  malakoff-rue etienne dolet                      1450451.0   

    Part de la station dans le trafic annuel de la ligne  \
0                                            0.050062      
1                                            0.039302      
2                                            0.038011      
3                                            0.037010      
4                                            0.036193      
5                                            0.047613      
6                                            0.030480      
7                                            0.021533      
8                                            0.053155      
9                                            0.025573      
10                                           0.024414      
12                                           0.047059      
13                                           0.018546      
14                                           0.023056      
15                                           0.021642      
16                                           0.017015      
17                                           0.016042      
18                                           0.055971      
19                                           0.049385      
20                                           0.031444      
21                                           0.030925      
22                                           0.013385      
23                                           0.026508      
24                                           0.025652      
25                                           0.010976      
26                                           0.070591      
27                                           0.033871      
28                                           0.056696      
29                                           0.027555      
30                                           0.020340      

    Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)  \
0                          



Tableau: ligne numéro[14]


Station  Trafic annuel dans la station  \
0           gare de lyon                      9546825.0   
2                  bercy                      1942106.0   
4              madeleine                      1776976.0   
5        porte de clichy                      2639248.5   
6     cour saint-emilion                      2985122.0   
7           bibliotheque                     11104474.0   
8             olympiades                      5214595.0   
9   mairie de saint-ouen                      2415405.0   
10         pont cardinet                      4168538.0   
11             pyramides                      1981349.0   
12     clichy saint-ouen                      3420852.0   

    Part de la station dans le trafic annuel de la ligne  \
0                                            0.202283      
2                                            0.041150      
4                                            0.037651      
5                                            0.055922      
6                                            0.063250      
7                                            0.235287      
8                                            0.110489      
9                                            0.051179      
10                                           0.088325      
11                                           0.041982      
12                                           0.072483      

    Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)  \
0                                         5298.853444                                                                 
2                                         1407.759105                                                                 
4                                         1850.791063                                                                 
5                                         1929.658213                                                                 
6                                         1084.386594                                                                 
7                                         2272.627107                                                                 
8                                         1895.432969                                                                 
9                                          846.124381                                                                 
10                                        1497.119048                                                                 
11                                        1287.978261                                                                 
12                                        1180.762691                                                                 

    Moyenne de la fréquentation journalière de chaque correspondance de la station en 2022, sur la période des JO  \
0                                         1766.284481                                                               
2                                          703.879553                                                               
4                                          616.930354                                                               
5                                          964.829106                                                               
6                                         1084.386594                                                               
7                                         2272.627107                                                               
8                                         1895.432969                                                               
9                                          423.062191                                                               
10                                        1497.119048                                                    